# Deep Agents

你可以把 [deepagents](https://docs.langchain.com/oss/python/deepagents/overview) 看作 LangChain 团队出品的 DeepResearch.

由于本 tutorial 写到现在已经又臭又长了，所以这里就 quickstart 一下。剩下的部分请大家去官网看吧！

In [1]:
# !pip install deepagents ddgs

**1）加载模型**

In [2]:
import os
from datetime import datetime
from dotenv import load_dotenv
from ddgs import DDGS
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from deepagents import create_deep_agent

# 加载模型配置
_ = load_dotenv()

# 加载模型
llm = ChatOpenAI(
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
    model="qwen3-coder-plus",
    temperature=0.7,
)

**2）创建搜索工具**

[DDGS](https://github.com/deedy5/ddgs) 是我找到的一款免费的、无需 `API_KEY`即可直接使用的网络搜索服务。我们用它替代官方推荐的 `Tavily`.

In [3]:
# 创建 ddgs 客户端
ddgs = DDGS()

def internet_search(
    query: str,
    max_results: int = 3,
) -> str:
    """
    使用互联网搜索指定关键词并返回格式化结果。

    Args:
        query: 搜索关键词或问题。
        max_results: 返回的最大结果条数。

    Returns:
        包含每条搜索结果的标题、摘要与链接的字符串。
    """
    results = list(
        ddgs.text(
            query=query,
            region="wt-wt",  # wt-wt zh
            timelimit='y',
            safesearch='off',  # moderate off
            page=1,
            backend='auto',
            max_results=max_results,
        )
    )

    content = ""
    for i, r in enumerate(results, 1):
        content += f"【结果 {i}】\n"
        content += f"标题: {r['title']}\n"
        content += f"摘要: {r['body']}\n"
        content += f"链接: {r['href']}\n\n"

    return content

In [4]:
# 测试
res = internet_search(query="美食 四川")
print(res)

【结果 1】
标题: 四川 小吃大起底，这10款 美食 ，你敢来试试不？ - 知乎
摘要: Mar 4, 2025 · 在美食的浩瀚宇宙中，四川小吃以其独特的麻辣鲜香，成为了无数食客心中的璀璨星辰。它们或藏于繁华街市，或隐于幽静小巷，却总能以那份独特的魅力，吸引着我们去探寻、去品味。今 …
链接: https://zhuanlan.zhihu.com/p/27740747873

【结果 2】
标题: 四川美食 TOP10大赏！没吃过这些，别说你懂川味|川菜|牛杂 ...
摘要: Jul 20, 2025 · 四川美食TOP10大赏！没吃过这些，别说你懂川味,川味,川菜,牛杂,小吃,汤底,火锅,龙抄手,担担面,四川省,四川美食说起四川美食，你脑子里是不是只有火锅和串串？那可就错过太多宝藏了！ …
链接: https://www.163.com/dy/article/K4TN45140556CWB7.html

【结果 3】
标题: 四川 十大名小吃指南丨看看有无你家乡的小吃呢？_腾讯新闻
摘要: May 14, 2024 · 除热辣辛香的火锅串串，这些小吃也是众多外地游客对四川念念不忘的理由。 今天就为大家分享除面条外四川各地有名的10种特色小吃，看...
链接: https://news.qq.com/rain/a/20240514A0AK8A00




**3）创建深度代理**

In [5]:
# System prompt to steer the agent to be an expert researcher
research_instructions = """你是一名资深研究员。你的工作是进行全面深入的研究，并撰写一份精炼的报告。

你可以使用互联网搜索工具作为获取信息的主要方式。

## `internet_search`

使用该工具对指定查询进行互联网搜索。你可以设置返回结果的最大数量。

对于该工具的 query 参数，每次最多输入 **2个** 关键词。且关键词之间必须用空格分开。若不遵守此条规定，工具将返回无意义内容。

正确用例：

- 南京
- 美食 四川

错误用例：

- 美食 四川 2025

注意，当关键词数量为2个的时候，必须将2个词中更重要的那个放在前面。
"""

@tool
def get_today_date() -> str:
    """获取今天的日期"""
    return datetime.now().strftime("%Y-%m-%d")

agent = create_deep_agent(
    model=llm,
    tools=[internet_search, get_today_date],
    system_prompt=research_instructions
)

**4）运行 Agent**

我们的问题是：

```
新上任的玻利维亚总统是谁？请介绍一下这位总统。
```

因为昨天（2025 年 11 月 8 日）玻利维亚总统 `罗德里戈・帕斯・佩雷拉` 刚刚上任，这条信息肯定不在预训练数据里。所以可以用这个问题验证 Deep Agents 是否真的联网了。

In [6]:
result = agent.invoke({"messages": [
    {"role": "user", "content": "新上任的玻利维亚总统是谁？请介绍一下这位总统。"}
]})

# 最终回复
print(result["messages"][-1].content)

### 新上任的玻利维亚总统介绍

#### 基本信息
- **姓名**：罗德里戈·帕斯 (Rodrigo Paz)
- **就职时间**：2025年11月8日
- **选举情况**：在2025年10月19日举行的总统选举第二轮投票中，帕斯以54.5%的得票率击败前总统豪尔赫·基罗加，当选为新一届玻利维亚总统。

#### 个人背景
虽然目前关于罗德里戈·帕斯的详细个人背景信息有限，但他显然是一位具有相当政治影响力的领导人。他的当选结束了玻利维亚一段时间的政治不确定性，使国家得以稳定发展。

#### 政治立场与政策影响
由于缺乏直接关于其政治立场的具体信息，我们可以从他当选后的表态和竞选承诺中推测：
1. **国内政策**
   - 帕斯在竞选期间强调了国家团结的重要性，主张弥合社会分歧，推动经济发展。
   - 他可能会关注改善民生、促进就业以及加强基础设施建设等领域。

2. **对外关系**
   - 在国际事务方面，帕斯曾表示希望保持与邻国及主要贸易伙伴的良好关系。
   - 对于中国等重要合作伙伴，他表达了继续深化双边合作的愿望，尤其是在经贸、矿业等领域。

#### 总结
罗德里戈·帕斯作为玻利维亚的新任总统，在2025年11月正式就职。尽管他的具体个人背景尚待更多披露，但从他在选举中的表现来看，他是一位致力于国家稳定的领导人。未来几年内，他领导下的玻利维亚有望在经济和社会发展方面取得积极进展，并维持与其他国家尤其是中国的友好合作关系。


In [7]:
# 思考过程
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

新上任的玻利维亚总统是谁？请介绍一下这位总统。
================================== Ai Message ==================================
Tool Calls:
  internet_search (call_6230adc734714e5492418aad)
 Call ID: call_6230adc734714e5492418aad
  Args:
    query: 玻利维亚 新总统 2025
    max_results: 3
================================= Tool Message =================================
Name: internet_search

【结果 1】
标题: 习近平同 玻 利 维 亚 总 统 阿尔塞就中 玻 建交40...
摘要: 发布时间： 2025 -07-09 15:45. 7月9日，国家主席习近平同 玻 利 维 亚 总 统 阿尔塞互致贺电，庆祝两国建交40周年。 习近平指出，中国和 玻 利 维 亚 是好朋友、好兄弟。
链接: https://hrss.tj.gov.cn/xinwenzixun/meitijujiao/202507/t20250709_6976406.html

【结果 2】
标题: 玻 利 维 亚 总 统 候选人竟嚷嚷：当选后，要搅黄中俄锂矿交易_凤凰网
摘要: 26.7%。 玻 利 维 亚 “自由”联盟候选人、前 总 统 豪尔赫·基罗加 法 新 社.阿尔塞于 2025 年5月14日宣布，为避免左翼选票进一步分裂，他将不再寻求连任。 同一天， 玻 利 维 亚 最高宪法法院裁定，莫拉莱斯因任期限制被禁止参加此次选举。
链接: https://news.ifeng.com/c/8m8VCY5yFW6

【结果 3】
标题: 玻 利 维 亚 将举行第二轮 总 统 选举_央广网
摘要: 玻 利 维 亚 将举行第二轮 总 统 选举. 2025 -08-18 10:47:17来源：央视 新 闻. 资料图. 玻 利